## Import Raw data for the Fermionic Operator
### Import the data from .csv
Steps here
1. Define the input file name
2. Code for the required 2body terms matrix element
3. Output a csv file that contain information for 2 pair promotion

In [9]:
## Manipulate the data into the form of i1, i2, f1, f2, V,

import pandas as pd

be8_df_source = pd.read_csv(r'new_Be8_2B_source.csv')
# be8_df_source.drop(be8_df_source.index[30:365],inplace=True) #,axis=0,  # ref https://www.freecodecamp.org/news/drop-list-of-rows-from-pandas-dataframe/#:~:text=You%20can%20delete%20a%20list,to%20the%20drop()%20method.&text=In%20this%20code%2C,be%20deleted%20from%20the%20dataframe

# be8_df_source.drop([364],axis=0,inplace=True)

be8_df = be8_df_source.copy()

## Static Helium core, level 1 and 2 not involved
## return the indexing to python style: aka start from 0 (orbitals)
be8_df['i1'] = be8_df['i1'] - 3 ; be8_df['i2'] = be8_df['i2'] - 3 ; # -3 because the data starts from 3
be8_df['f1'] = be8_df['f1'] - 3 ; be8_df['f2'] = be8_df['f2'] - 3 ;

## Defining orbital indeces
be8_df['i1'] = be8_df['i1'].add(be8_df['iso'].mul(8) -8) ; #define respective index for n and p
be8_df['i2'] = be8_df['i2'].add(be8_df['iso.1'].mul(8) -8) ; 
be8_df['f1'] = be8_df['f1'].add(be8_df['iso.2'].mul(8) -8) ; 
be8_df['f2'] = be8_df['f2'].add(be8_df['iso.3'].mul(8) -8) ; 

# be8_df.drop(['iso','iso.1','iso.2','iso.3'],axis=1,inplace=True)

be8_df

,i1,iso,i2,iso.1,f1,iso.2,f2,iso.3,V
0,0,1,1,1,0,1,1,1,-10.548557
1,0,1,1,1,0,1,7,1,-1.097954
2,0,1,1,1,1,1,6,1,1.097954
3,0,1,1,1,2,1,3,1,-4.407839
4,0,1,1,1,4,1,5,1,6.129595
...,...,...,...,...,...,...,...,...,...
675,14,2,15,2,8,2,14,2,-0.449919
676,14,2,15,2,9,2,15,2,0.449919
677,14,2,15,2,10,2,11,2,-5.653955
678,14,2,15,2,12,2,13,2,2.378961


In [10]:
### To drop duplicates and limit only up transfer j>i where i is init j is fina

from itertools import combinations
combo_1 = list(combinations(range(0,16),2))           ## Filter for same type nucleons dont occupy same orbitals
degenerate_levels = [(0,1),(2,3),(4,5),(6,7),(8,9),(10,11),(12,13),(14,15)]
init_HF_levels = [0,1,8,9]
tz1_pairing = []
list08 = list(range(0,8)); list816 = list(range(8,16))

## Filtering for the 12 pairs promotions
for init in combo_1:
    for fina in combo_1:
        
        ## filter if the particles stay in their own set of orbitals
        if ((init[0] in list08) == (fina[0] in list08) and
            (init[1] in list08) == (fina[1] in list08) and
            (init[0] in list816) == (fina[0] in list816) and
            (init[1] in list816) == (fina[1] in list816)):

            ## filter to limit to allow only pair promotion
            if ((init in degenerate_levels) and (fina in degenerate_levels)):
                if (init[0] in init_HF_levels) and (init[1] in init_HF_levels): ## Filter allow for state starting from 0167
                    tz1_pairing.append((init,fina))
                    if init == fina:
                        pass
                    else:
                        tz1_pairing.append((fina,init))
                else: 
                    pass
            else:
                pass
        else: 
            pass

## Filtering for the Tz = 0 pair
### rules: they must have S = 0
neutron = {}
proton = {}
neutron[0] = 1
neutron[1] = -1
neutron[2] = -1
neutron[3] = 1
neutron[4] = -1
neutron[5] = 1
neutron[6] = 1
neutron[7] = -1

proton[8] = 1
proton[9] = -1
proton[10] = 1
proton[11] = -1
proton[12] = 1
proton[13] = -1
proton[14] = -1
proton[15] = 1

## combo2: combination that has S=0 
combo2 = []
for neu in neutron:
    for pro in proton:
        S = neutron[neu]+proton[pro]
        if S == 0 : 
            combo2.append((neu,pro))

## combo3: combination that has pairs in same level
combo3 = []
for i in [0,1]:
    for j in [8,9]:
        combo3.append((i,j))
for i in [2,3]:
    for j in [10,11]:
        combo3.append((i,j))
for i in [4,5]:
    for j in [12,13]:
        combo3.append((i,j))
for i in [6,7]:
    for j in [14,15]:
        combo3.append((i,j))


## list of overlaps between combo3 and combo2 And account for pairings that start from 0,1,8,9
tz0_pairing = []
for init in combo3:
    for fina in combo3:
        ## Combo2 contains all pairs that obey S=0
        ## Filter for S=0 pairs
        if (init in combo2) and (fina in combo2):
            if (init[0] in init_HF_levels) and (init[1] in init_HF_levels): ## Filter allow for state starting from 0167
                tz0_pairing.append((init,fina))
                if init == fina:
                    pass
                else:
                    tz0_pairing.append((fina,init))
            else: 
                pass
        else:
            pass
        
pairing_list = tz1_pairing + tz0_pairing
print(len(pairing_list))    
print(tz1_pairing)
# print(tz0_pairing)


44
[((0, 1), (0, 1)), ((0, 1), (2, 3)), ((2, 3), (0, 1)), ((0, 1), (4, 5)), ((4, 5), (0, 1)), ((0, 1), (6, 7)), ((6, 7), (0, 1)), ((8, 9), (8, 9)), ((8, 9), (10, 11)), ((10, 11), (8, 9)), ((8, 9), (12, 13)), ((12, 13), (8, 9)), ((8, 9), (14, 15)), ((14, 15), (8, 9))]


In [11]:
# Construct a drop list to drop all those which are not in the pairing list (interaction that we are not interested)
drop_list = []

for line_number, (index, row) in enumerate(be8_df.iterrows()):
#     print(line_number)
    init_1 = int(row['i1']); init_2 = int(row['i2']);
    fina_1 = int(row['f1']); fina_2 = int(row['f2']);
    filterr = ((init_1,init_2),(fina_1,fina_2))
    
    if (filterr in pairing_list):
        pass
    else:
        drop_list.append(line_number)
        
## Drop all the from the drop list
print(len(drop_list))
# print(drop_list)
be8_df.drop(drop_list,axis=0,inplace=True)
be8_df.reset_index(drop=True,inplace=True)

## to save the dataframe into a csv file
# be8_df.to_csv("be8_pairing_T=1")


be8_df


638


,i1,iso,i2,iso.1,f1,iso.2,f2,iso.3,V
0,0,1,1,1,0,1,1,1,-10.548557
1,0,1,1,1,2,1,3,1,-4.407839
2,0,1,1,1,4,1,5,1,6.129595
3,0,1,1,1,6,1,7,1,-3.565626
4,0,1,9,2,0,1,9,2,-5.235812
5,0,1,9,2,1,1,8,2,5.235812
6,0,1,9,2,2,1,10,2,-2.183231
7,0,1,9,2,3,1,11,2,2.183231
8,0,1,9,2,4,1,12,2,3.014289
9,0,1,9,2,5,1,13,2,-3.014289


In [12]:
# the variable name above will be changed in the future
be8_df.to_csv("obs_twobody-new_be8.csv", index=False)

In [ ]:
# archived codes
# #             ## filter allow for the particles only go up 
# #             if (fina[0]>=init[0] and fina[1]>=init[1]):
# #             else:
# #                 pass



# ## Filtering for the Tz = 0 pair
# ### rules: they must have S = 0
# neutron = {}
# proton = {}
# neutron[0] = 1
# neutron[1] = -1
# neutron[2] = -1
# neutron[3] = 1
# neutron[4] = 1
# neutron[5] = -1

# proton[6] = 1
# proton[7] = -1
# proton[8] = 1
# proton[9] = -1
# proton[10] = -1
# proton[11] = 1
# combo2 = []
# for neu in neutron:
#     for pro in proton:
#         S = neutron[neu]+proton[pro]
#         if S == 0 : 
#             combo2.append((neu,pro))

# ## pairs that is of the same energy level (only nuclear)
# filter_same_level = []
# list1 = [0,1,6,7]
# list2 = [2,3,8,9]
# list3 = [4,5,10,11]

# list1_ = list(combinations(list1,2))
# list2_ = list(combinations(list2,2))
# list3_ = list(combinations(list3,2))
# filter_same_level = list1_ + list2_ + list3_
# print(filter_same_level)
# for init in combo2: 
#     for fina in combo2: 
#         if (fina[0]>=init[0] and fina[1]>=init[1]):  ## Filter for up promotion only.
#             if (fina in filter_same_level) and (init in filter_same_level): ## Filter out not pair (include only promotion of n and p to same nuclear energy)
#                 pairing_list.append((init,fina))
#             else:
#                 pass
#         else:
#             pass
# #         if (init[0] in init_HF_levels) and (init[1] in init_HF_levels): ## Filter allow for state starting from 0167
# #         else: 
# #             pass